In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import mean_squared_error
from src.utils.finance_metrics import annualized_return
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

In [2]:
data_path = r'data/processed'
data_filenames = os.listdir(data_path)

models = {}
feature_scalers = {}
target_scalers = {}
for data_file in data_filenames:
    pairs = data_file[:-4].split('-')
    pair = (pairs[0], pairs[1])

    pair_df = pd.read_pickle(os.path.join(data_path, data_file))
    X = pair_df.loc["2000-01-01":"2014-12-31", :].drop(columns="Return Diff (t+1)")
    y = pair_df.loc["2000-01-01":"2014-12-31", ["Return Diff (t+1)"]]
    X_test = pair_df.loc["2015-01-01":, :].drop(columns="Return Diff (t+1)")
    y_test = pair_df.loc["2015-01-01":, ["Return Diff (t+1)"]]

    # scale with QuantileTransformer and StandardScaler
    X_scaler = QuantileTransformer()
    X_scaler.fit(X); feature_scalers[pair] = X_scaler
    X_train = pd.DataFrame(X_scaler.transform(X), index=X.index, columns=X.columns)
    X_test = pd.DataFrame(X_scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    y_scaler = StandardScaler()
    y_scaler.fit(y); target_scalers[pair] = y_scaler
    y_train = pd.DataFrame(y_scaler.transform(y), index=y.index, columns=y.columns)
    y_test = pd.DataFrame(y_scaler.transform(y_test), index=y_test.index, columns=y_test.columns)
    assert False == any([any(arr) for arr in np.array(np.isinf(X_train))])
    assert False == any([any(arr) for arr in np.array(np.isnan(X_train))])
    assert False == any([any(arr) for arr in np.array(np.isinf(X_test))])
    assert False == any([any(arr) for arr in np.array(np.isnan(X_test))])
    assert False == any([any(arr) for arr in np.array(np.isinf(y_train))])
    assert False == any([any(arr) for arr in np.array(np.isnan(y_train))])
    assert False == any([any(arr) for arr in np.array(np.isinf(y_test))])
    assert False == any([any(arr) for arr in np.array(np.isnan(y_test))])

    split = PredefinedSplit(test_fold=[0 if v else -1 for v in X_train.index < '2012-01-01'])
    models[(pairs[0], pairs[1])] = {
        'Linear Regression': GridSearchCV(estimator=ElasticNet(max_iter=100000, tol=0.0004), param_grid={
            'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]
        }, cv=split, scoring="explained_variance", n_jobs=-1),
        'Support Vector Machine': GridSearchCV(estimator=SVR(cache_size=1000), param_grid={
            'C': [10**n for n in range(-3, 4)]
        }, cv=split, scoring="explained_variance", n_jobs=-1),
        'Random Forest': GridSearchCV(estimator=RandomForestRegressor(bootstrap=True), param_grid={
            "n_estimators": [n for n in range(50, 150, 25)]
        }, cv=split, scoring="explained_variance", n_jobs=-1),
        'Adaptive Boost': GridSearchCV(estimator=AdaBoostRegressor(), param_grid={
            "base_estimator": [DecisionTreeRegressor(max_depth=1), DecisionTreeRegressor(max_depth=3)],
            "n_estimators": [n for n in range(50, 250, 50)],
            "loss": ["linear", "exponential"]
        }, cv=split, scoring="explained_variance", n_jobs=-1),
        'Gradient Boost': GridSearchCV(estimator=GradientBoostingRegressor(loss="huber"), param_grid={
            "n_estimators": [n for n in range(50, 250, 50)]
        }, cv=split, scoring="explained_variance", n_jobs=-1),
        'Neural Net': GridSearchCV(estimator=MLPRegressor(solver="lbfgs", max_iter=1000000), param_grid={
            "hidden_layer_sizes": [(h1, h2)
                                   for h1 in range(100, 200, 50)
                                   for h2 in range(25, h1//2, 25)]
        }, cv=split, scoring="explained_variance", n_jobs=-1)
    }

    print(f"==================================================\n"
          f"Results for pair {pair}:")
    pred_average = None
    for model_type, model in models[pair].items():
        model.fit(X_train, np.array(y_train).ravel())
        pred = model.predict(X_test)
        if model_type != "Linear Regression":
            if pred_average is None: pred_average = pred / 5
            else: pred_average = pred_average + np.array(pred / 5)
        mse = mean_squared_error(np.array(y_test), np.array(pred).ravel())
        print(f"- Model: {model_type}\n"
              f"  - best parameters: {model.best_params_}\n"
              f"  - MSE: {mse:.06f}\n"
              f"--------------------------------------------------")
    mse = mean_squared_error(np.array(y_test), np.array(pred_average).ravel())
    print(f"- Ensemble of all models\n"
          f"  - MSE: {mse:.06f}\n"
          f"--------------------------------------------------")
    print(f"==================================================\n")



Results for pair ('BAC', 'WFC'):
- Model: Linear Regression
  - best parameters: {'l1_ratio': 0.1}
  - MSE: 0.295929
--------------------------------------------------
- Model: Support Vector Machine
  - best parameters: {'C': 100}
  - MSE: 0.127596
--------------------------------------------------
- Model: Random Forest
  - best parameters: {'n_estimators': 75}
  - MSE: 0.000072
--------------------------------------------------
- Model: Adaptive Boost
  - best parameters: {'base_estimator': DecisionTreeRegressor(max_depth=3), 'loss': 'exponential', 'n_estimators': 100}
  - MSE: 0.042979
--------------------------------------------------
- Model: Gradient Boost
  - best parameters: {'n_estimators': 200}
  - MSE: 0.000104
--------------------------------------------------
- Model: Neural Net
  - best parameters: {'hidden_layer_sizes': (100, 25)}
  - MSE: 0.004547
--------------------------------------------------
- Ensemble of all models
  - MSE: 0.008218
-----------------------------

KeyboardInterrupt: 